<a href="https://colab.research.google.com/github/d-m-bailey/ihp-mpw-be/blob/bandgap_ref_cmos/TO_202504.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backend Verification for the iHP Open MPW Shuttles

[D. Mitch Bailey](https://www.linkedin.com/in/mitch-bailey-cvc/), [ShuhariSystem](https://www.shuharisystem.com/)

# Overview
This notebook performs backend LVS (with [klayout](https://github.com/KLayout/klayout) or [magic](https://github.com/RTimothyEdwards/magic)/[netgen](https://github.com/RTimothyEdwards/netgen)), soft-connection checks (with magic/netgen) and/or reliability verfication (with [CVC-RV](https://github.com/d-m-bailey/cvc)) with open source EDA software and systems.

# iHP Open-MPW Data
The 17 submissions to iHp open mpw shuttle for April 2025 listed below are located [here](https://github.com/IHP-GmbH/TO_Apr2025).

* 160GHz_LNA
* 40_GHZ_LOW_NOISE_TIA
* 6502-cpu
* 97_GHZ_LINEAR_TIA
* DC_to_130_GHz_TIA
* GPS_LNA
* Greyhound
* Mixer5GHz
* PA_180GHz
* TTIHP0p2
* TTIHP25a
* VCO_130nm_LSI
* active_L_VCOs
* ascon
* bandgap_ref_cmos
* elemrv-n
* i2c-gpio-expander

Create a file with the initial environment variables.


In [11]:
%%writefile /content/env
export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export UPRJ_ROOT=/content/data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025
if [[ -f $UPRJ_ROOT/project_env ]]; then
  cat $UPRJ_ROOT/project_env
  source $UPRJ_ROOT/project_env
fi


Overwriting /content/env


# Program installation.
Only needs to be executed once.

This step sets up the pdk and installs magic, klayout, netgen and cvc_rv.

Runtime: 3-4 minutes.

In [12]:
%%shell
cd
cat /content/env
source /content/env

lsb_release -a 2>/dev/null

if [[ ! -d $PDK_ROOT ]]; then
  echo "==> Installing ciel..."
  pip install ciel
fi
ciel enable --pdk $PDK $PDK_COMMIT
# patch for mimcap and antenna error in netgen setup file from pdk
sed -i '/lsearch .cells2/,/circuit2/s/circuit1 .dev/circuit2 $dev/' $PDK_ROOT/$PDK/libs.tech/netgen/ihp-sg13g2_setup.tcl
echo "==> Using pdk $PDK commit $PDK_COMMIT (patched)
"

if [[ ! -d extra_be_checks ]]; then
  echo "==> Downloading extra_be_checks $EXTRA_CHECK_COMMIT"
  rm -rf extra_be_checks
  git clone https://github.com/d-m-bailey/extra_be_checks.git -b $EXTRA_CHECK_COMMIT
fi
echo "==> Using extra_be_checks commit $(cd extra_be_checks; git rev-parse HEAD)
"

if ! command -v netgen; then
  echo "==> Downloading and installing netgen $NETGEN_COMMIT"
  git clone https://github.com/RTimothyEdwards/netgen.git --depth=1 -b $NETGEN_COMMIT
  cd netgen
  ./configure --prefix=$LOCAL_INSTALL
  make
  make install
  cd
fi
echo "==> Using netgen version $(netgen -batch | awk '/Netgen/ {print $2}')
"

if ! command -v magic; then
  echo "==> Downloading and installing magic $MAGIC_COMMIT"
  git clone https://github.com/RTimothyEdwards/magic.git --depth=1 -b $MAGIC_COMMIT
  cd magic
  ./configure --prefix=$LOCAL_INSTALL
  make
  make install
  cd
fi
echo "==> Using magic version $(magic -dnull -noc --version)
"

if ! command -v klayout; then
  echo "==> Downloading and installing klayout $KLAYOUT_DEB for $KLAYOUT_DISTRO"
  wget -P /root https://www.klayout.org/downloads/$KLAYOUT_DISTRO/$KLAYOUT_DEB
  sudo apt install /root/$KLAYOUT_DEB
  pip install docopt
  pip install klayout
  cd
fi
echo "==> Using $(klayout -v)
"

if ! command -v cvc_rv; then
  echo "==> Downloading and installing cvc_rv $CVC_COMMIT"
  sudo apt install autopoint bison flex
  git clone https://github.com/d-m-bailey/cvc --depth=1 -b $CVC_COMMIT
  cd cvc
  autoreconf -vif
  ./configure --prefix=$LOCAL_INSTALL --disable-nls
  make
  make install
  cd
fi
echo "==> Using $(cvc_rv -v)
"

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export UPRJ_ROOT=/content/data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025
if [[ -f $UPRJ_ROOT/project_env ]]; then
  cat $UPRJ_ROOT/project_env
  source $UPRJ_ROOT/project_env
fi
export PROJECT=bandgap_ref_cmos
export LAYOUT_TOP=full_bandgap
export GDSFILE=design_data/gds/FMD_QNC_15_WeakInvBGR.gds
export KLAYOUT_CDL=/content/data/bandgap_reference.cdl
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy
⠋ Enabling version ddb601a4a4473163e1ed6df416b885df18b4ac03…
Version ddb601a4a4473163e1ed6df416b885df18b4ac03 enabled for the ihp-sg13

# Clone the design repo and list the gds, spice and verilog files.

Run to refresh the design data.
The file list is also saved in /content/filelist.txt.

Runtime: 1-2 minutes.

In [13]:
%%shell
cat /content/env
source /content/env
rm -rf data
git clone https://github.com/IHP-GmbH/$MPW.git data
cd data
# uncompress gds, spice, and verilog files
find . \( -name "*.gds.gz" -o -name "*.spice.gz" -o -name "*.v.gz" \) -execdir gunzip -v {} \;
find . \( -name "*.gds.zip" -o -name "*.spice.zip" -o -name "*.v.zip" \) -execdir 7z x {} \;
# Create a file listing the size, project and gds file name.
# Only list files that end with .gds (in gds subdirectories), .spice, or .v.
tee /content/filelist.txt <<EOF
   Size Project                   File
------- ------------------------- ---------------------------------------------
EOF
ls -s $(find . \( -name "*.gds" -path "*/gds/*" \) -o -name "*.spice" -o -name "*.v") |
  sed -e 's,\./,,' -e 's,/, ,' |
  awk '{printf "%7d %-25s %s\n", $1, $2, $3}' |
  tee -a /content/filelist.txt

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export UPRJ_ROOT=/content/data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025
if [[ -f $UPRJ_ROOT/project_env ]]; then
  cat $UPRJ_ROOT/project_env
  source $UPRJ_ROOT/project_env
fi
export PROJECT=bandgap_ref_cmos
export LAYOUT_TOP=full_bandgap
export GDSFILE=design_data/gds/FMD_QNC_15_WeakInvBGR.gds
export KLAYOUT_CDL=/content/data/bandgap_reference.cdl
Cloning into 'data'...
remote: Enumerating objects: 3270, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 3270 (delta 154), reused 149 (delta 143), pac

# Create setup for bandgap_ref_cmos


Set the project, top cell and gds file names.

In [14]:
%%shell
source /content/env
cat > $UPRJ_ROOT/project_env <<EOF
export PROJECT=bandgap_ref_cmos
export LAYOUT_TOP=full_bandgap
export GDSFILE=design_data/gds/FMD_QNC_15_WeakInvBGR.gds
export WORK_ROOT=$UPRJ_ROOT/$PROJECT/work
export SPICE_FILE=$WORK_ROOT/$LAYOUT_TOP.spice
export KLAYOUT_CDL=$WORK_ROOT/$LAYOUT_TOP.cdl
EOF
cat $UPRJ_ROOT/project_env
source $UPRJ_ROOT/project_env
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT

export PROJECT=bandgap_ref_cmos
export LAYOUT_TOP=full_bandgap
export GDSFILE=design_data/gds/FMD_QNC_15_WeakInvBGR.gds
export WORK_ROOT=/content/data//work
export SPICE_FILE=/.spice
export KLAYOUT_CDL=/.cdl


Create the modified magic source netlist.

In [15]:
%%shell
cat /content/env
source /content/env
cat > $SPICE_FILE <<EOF
** sch_path: /Users/home/IHP/TO_Apr2025/bandgap_ref_cmos/design_data/xschem/part_2_full_bgr/bandgap_reference.sch
.subckt full_bandgap vdd VBG vss iout
*.PININFO vplus:B v-:B Vo1:B VBG:O
XM8 net1 v- vss vss sg13_lv_nmos l=10u w=150n ng=1 m=1
XM6 net1 net1 vdd vdd sg13_lv_pmos l=1u w=1u ng=1 m=1
XM7 net2 net1 vdd vdd sg13_lv_pmos l=1u w=1u ng=1 m=1
XM9 v- net2 vdd vdd sg13_lv_pmos l=4u w=200n ng=1 m=1
XM1 v- vplus vss vss sg13_lv_nmos l=5u w=7.14u ng=4 m=1
XM2 net3 net3 vss vss sg13_lv_nmos l=5u w=21u ng=8 m=1
XM3 v- Vo1 vdd vdd sg13_lv_pmos l=5u w=15u ng=8 m=1
XM4 vplus Vo1 vdd vdd sg13_lv_pmos l=5u w=15u ng=8 m=1
XM5 VBG Vo1 vdd vdd sg13_lv_pmos l=5u w=16u ng=8 m=1
*XC3 VBG vss cap_cmim w=72.965e-6 l=72.965e-6 m=1
XC3 vdd vss cap_cmim w=5e-6 l=5e-6 m=15
XR3 net3 vplus rppd w=0.5e-6 l=194.345e-6 m=1 b=0
*XR1 vss vplus rppd w=0.6e-6 l=194.345e-6 m=1 b=0
XR1_1 vss mid rppd w=03.0e-6 l=38.645e-6 m=1 b=0
XR1_2 mid vplus rppd w=0.5e-6 l=154.58e-6 m=1 b=0
*XR2 vss VBG rppd w=0.5e-6 l=192.395e-6 m=1 b=0
XR2 vss VBG rppd w=0.71e-6 l=270.515e-6 m=1 b=0
XC1 net2 vss cap_cmim w=18.195e-6 l=18.195e-6 m=1
x1 vdd iout vplus v- Vo1 vss two_stage_OTA
.ends

* expanding   symbol:  part_1_OTA/two_stage_OTA.sym # of pins=6
** sym_path: /Users/home/IHP/TO_Apr2025/bandgap_ref_cmos/design_data/xschem/part_1_OTA/two_stage_OTA.sym
** sch_path: /Users/home/IHP/TO_Apr2025/bandgap_ref_cmos/design_data/xschem/part_1_OTA/two_stage_OTA.sch
.subckt two_stage_OTA vdd iout vplus v- vout vss
*.PININFO v-:B vplus:B vss:B vdd:B iout:B vout:B
XM4 net3 net1 vss vss sg13_lv_nmos l=9.75u w=720n ng=1 m=1
XM3 net1 net1 vss vss sg13_lv_nmos l=9.75u w=720n ng=1 m=1
XM1 net1 v- net2 vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=2
XM_dummy net1 vdd vdd vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=2
XM2 net3 vplus net2 vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=2
XM_dummy2 vdd vdd net2 vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=4
XM_dummy3 net3 vdd vdd vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=2
XM5 net2 iout vdd vdd sg13_lv_pmos l=1.95u w=5.3u ng=1 m=1
XM7 vout iout vdd vdd sg13_lv_pmos l=2.08u w=75u ng=8 m=1
XM6 vout net3 vss vss sg13_lv_nmos l=9.75u w=28.8u ng=4 m=1
XM9 iout iout vdd vdd sg13_lv_pmos l=2.08u w=75u ng=8 m=1
XC2 net3 vout cap_cmim w=22.295e-6 l=22.295e-6 m=1
.ends
EOF

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export UPRJ_ROOT=/content/data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025
if [[ -f $UPRJ_ROOT/project_env ]]; then
  cat $UPRJ_ROOT/project_env
  source $UPRJ_ROOT/project_env
fi
export PROJECT=bandgap_ref_cmos
export LAYOUT_TOP=full_bandgap
export GDSFILE=design_data/gds/FMD_QNC_15_WeakInvBGR.gds
export WORK_ROOT=/content/data//work
export SPICE_FILE=/.spice
export KLAYOUT_CDL=/.cdl


Create lvs_config.json and update the LVS_SPICE_FILES and LVS_VERILOG_FILES for every design.

Update the other parameters as needed.

In [16]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "$LAYOUT_TOP",
  "TOP_LAYOUT": "$LAYOUT_TOP",
  "EXTRACT_FLATGLOB": [ "" ],
  "EXTRACT_ABSTRACT": [ "" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "" ],
  "LVS_SPICE_FILES": [
    "$SPICE_FILE"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/$GDSFILE"
}

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export UPRJ_ROOT=/content/data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025
if [[ -f $UPRJ_ROOT/project_env ]]; then
  cat $UPRJ_ROOT/project_env
  source $UPRJ_ROOT/project_env
fi
export PROJECT=bandgap_ref_cmos
export LAYOUT_TOP=full_bandgap
export GDSFILE=design_data/gds/FMD_QNC_15_WeakInvBGR.gds
export WORK_ROOT=/content/data//work
export SPICE_FILE=/.spice
export KLAYOUT_CDL=/.cdl
/bin/bash: line 24: warning: here-document at line 3 delimited by end-of-file (wanted `EOF')


Write cvcrc file

In [17]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $KLAYOUT_CDL
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/ihp-sg13g2.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power
CVC_REPORT_FILE = cvc.log
EOF

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export UPRJ_ROOT=/content/data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025
if [[ -f $UPRJ_ROOT/project_env ]]; then
  cat $UPRJ_ROOT/project_env
  source $UPRJ_ROOT/project_env
fi
export PROJECT=bandgap_ref_cmos
export LAYOUT_TOP=full_bandgap
export GDSFILE=design_data/gds/FMD_QNC_15_WeakInvBGR.gds
export WORK_ROOT=/content/data//work
export SPICE_FILE=/.spice
export KLAYOUT_CDL=/.cdl


Create the klayout CDL file from the magic source netlist.

In [18]:
%%shell
source /content/env
sed -e '/^X[MRC]/s/.//' $SPICE_FILE > $KLAYOUT_CDL


export PROJECT=bandgap_ref_cmos
export LAYOUT_TOP=full_bandgap
export GDSFILE=design_data/gds/FMD_QNC_15_WeakInvBGR.gds
export WORK_ROOT=/content/data//work
export SPICE_FILE=/.spice
export KLAYOUT_CDL=/.cdl


# Results for bandgap_ref_cmos

*   Pad text is on metal7 layer. This is not recognized on pads in magic. Either move the text to a region outside the pad, change the text layer to the PADID layer (41/0), or modify the magic extraction rules to recognize metal7 text on pads.
*   Klayout extraction rules generate n/ptap devices for layout taps annotated with "well"/"sub!" text. The schematic does not include these devices. Either add the devices to the schematic or remove the "well" and "sub!" text from the layout.
*   Ports do not match. Layout ports are vss, vdd, VBG, and iout. Netlist ports are GND, vdd, v+, v-, Vo1, and VBG. iout in the schematic is currently shorted to ground through an ammeter. Suggest removing v+, v-, Vo1 ports in schematic while adding iout port. Suggest changing GND from a global net to a port named vss in the schematic to match the layout.
* Layout has 3 dummy pmos (gate and source connected to vdd) not found in the schematic. Suggest adding them to the schematic. Suggest adding the following to `two_stage_OTA`
```
XM_dummy net1 vdd vdd vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=2
XM_dummy2 vdd vdd net2 vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=4
XM_dummy3 net3 vdd vdd vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=2
```
* cap_cmim connected between VBG and vss in the schematic is connected between vdd and vss in the layout. The schematic size is 72.965 x 72.965 but the layout size is 5x5 m=15.
* R2 schematic size is 0.5/192.395 but layout is 0.71/38.645x7
* R1 schematic size is 0.6/194.345 but layout consists of resistors of different widths. 3.0/38.645 and 0.5/38.645x4. Currently netgen and klayout only reduce resistors with the same widths.



# Run BE checks

In [19]:
%%shell
cat /content/env
source /content/env
$LVS_ROOT/run_be_checks $WORK_ROOT/lvs_config.json
#sed -i 's/^extract/# extract/' $PDK_ROOT/$PDK/libs.tech/klayout/tech/lvs/rule_decks/tap_extraction.lvs
# ignore rule that creates separate taps for labeled wells
sed -i -e '/^well_patt/s/"/"-/' -e '/^sub_patt/s/"/"-/' $PDK_ROOT/$PDK/libs.tech/klayout/tech/lvs/rule_decks/general_derivations.lvs
python3 $PDK_ROOT/$PDK/libs.tech/klayout/tech/lvs/run_lvs.py --layout=$UPRJ_ROOT/$PROJECT/$GDSFILE \
  --netlist=$KLAYOUT_CDL \
  --run_dir=$WORK_ROOT/klayout \
  --topcell=$LAYOUT_TOP \
  --run_mode=deep \
  --spice_comments \
  --no_simplify \
  --combine_devices \
  --top_lvl_pins

  # Run CVC(RV)
  cvc_rv $WORK_ROOT/cvcrc


export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export UPRJ_ROOT=/content/data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025
if [[ -f $UPRJ_ROOT/project_env ]]; then
  cat $UPRJ_ROOT/project_env
  source $UPRJ_ROOT/project_env
fi
export PROJECT=bandgap_ref_cmos
export LAYOUT_TOP=full_bandgap
export GDSFILE=design_data/gds/FMD_QNC_15_WeakInvBGR.gds
export WORK_ROOT=/content/data//work
export SPICE_FILE=/.spice
export KLAYOUT_CDL=/.cdl
Loading LVS environment from /content/data/work/lvs_config.json

TOP SOURCE: full_bandgap
SOURCE FILE(S): /.spice
TOP LAYOUT: full_bandgap
LAYOUT FILE: /content/data/bandgap_ref_cmos/desi

# View verification logs

In [20]:
from google.colab import files

files.view('/content/data/work/soft.log')
files.view('/content/data/work/lvs.log')
files.view('/content/data/work/cvc.log')

FileNotFoundError: Cannot find file: /content/data/work/soft.log

# View soft verification report

In [ ]:
from google.colab import files

files.view('/content/data/work/soft.report')

# View LVS report

In [ ]:
from google.colab import files

files.view('/content/data/work/lvs.report')

# View CVC report

In [ ]:
from google.colab import files

files.view('/content/data/work/cvc.error')